In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

from Utils import report, plot_roc_curve, plot_lost_history, confusion_matrix, append_to_binary_results, _confusion_matrix

In [7]:
df_houses = pd.read_csv("../Datasets/houses_to_rent_fiap.csv", delimiter=";")

In [8]:
df_houses.head()

,city,area,rooms,bathroom,parking spaces,floor,animal,furniture,hoa (R$),property tax (R$),fire insurance (R$),rent amount (R$)
0,São Paulo,70,2,1,1,7,acept,furnished,2065,211,42,3300
1,São Paulo,320,4,4,0,20,acept,not furnished,1200,1750,63,4960
2,Porto Alegre,80,1,1,1,6,acept,not furnished,1000,0,41,2800
3,Porto Alegre,51,2,1,0,2,acept,not furnished,270,22,17,1112
4,São Paulo,25,1,1,0,1,not acept,not furnished,0,25,11,800


In [10]:
df_houses.shape

(10692, 12)

In [11]:
df_houses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10692 entries, 0 to 10691
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   city                 10692 non-null  object
 1   area                 10692 non-null  int64 
 2   rooms                10692 non-null  int64 
 3   bathroom             10692 non-null  int64 
 4   parking spaces       10692 non-null  int64 
 5   floor                10692 non-null  object
 6   animal               10692 non-null  object
 7   furniture            10692 non-null  object
 8   hoa (R$)             10692 non-null  int64 
 9   property tax (R$)    10692 non-null  int64 
 10  fire insurance (R$)  10692 non-null  int64 
 11  rent amount (R$)     10692 non-null  int64 
dtypes: int64(8), object(4)
memory usage: 1002.5+ KB


In [14]:
df_houses.isna().sum()

city                   0
area                   0
rooms                  0
bathroom               0
parking spaces         0
floor                  0
animal                 0
furniture              0
hoa (R$)               0
property tax (R$)      0
fire insurance (R$)    0
rent amount (R$)       0
dtype: int64